Packages

In [1]:
import os
import numpy
import pandas
import tqdm
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline
import sys
sys.path.append('../scripts/')
from read_utils import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

22/09/21 10:46:40 WARN Utils: Your hostname, SukiXuudeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 100.70.0.56 instead (on interface en0)
22/09/21 10:46:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/21 10:46:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/21 10:46:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/21 10:46:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/21 10:46:43 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
file_name_list = sorted(os.listdir('../data/curated/data'))

In [4]:
sdf = read_file(spark, 'data', '../data/curated/')

|> Loading File...


|> Loading Finished!
22/09/21 10:46:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0------------------------------------------------------------------------
 order_id        | 00085ae9-13bb-477b-887d-72f53da720b2                          
 user_id         | 1215                                                          
 merchant_abn    | 63685007785                                                   
 dollar_value    | 160.82357068263116                                            
 merchant_name   | Augue Eu Tempor Associates                                    
 tags            | cable, satellite, and other pay television and radio services 
 take_rate       | 0.38                                                          
 type            | e                                                             
 consumer_id     | 25421                                                         
 name            | Laurie Sanchez                                                
 address         | 8509 Todd Brooks Apt. 348                                     
 state          

In [17]:
# type
type_indexer = StringIndexer(inputCol = "type", outputCol = "typeIndex", handleInvalid="skip")

# gender
gender_indexer = StringIndexer(inputCol = "gender", outputCol = "genderIndex", handleInvalid="skip")

# tag
tag_indexer = StringIndexer(inputCol = "tags",outputCol = "tagIndex", handleInvalid="skip")
onehotencoder_tag_vector = OneHotEncoder(inputCol = "tagIndex", outputCol = "tag_vec")

#Create pipeline and pass all stages
pipeline = Pipeline(stages=[gender_indexer, 
                            type_indexer,
                            tag_indexer,
                            onehotencoder_tag_vector])

In [19]:
sdf.show(5)

+--------------------+-------+------------+------------------+--------------------+--------------------+---------+----+-----------+---------------+--------------------+-----+--------+------+---+---+---+---+---+---+---+---+---+------+---------------+--------------+
|            order_id|user_id|merchant_abn|      dollar_value|       merchant_name|                tags|take_rate|type|consumer_id|           name|             address|state|postcode|gender|AUS|NSW|VIC|QLD| SA| WA|TAS| NT|ACT|target|New cases / day|order_datetime|
+--------------------+-------+------------+------------------+--------------------+--------------------+---------+----+-----------+---------------+--------------------+-----+--------+------+---+---+---+---+---+---+---+---+---+------+---------------+--------------+
|00085ae9-13bb-477...|   1215| 63685007785|160.82357068263116|Augue Eu Tempor A...|cable, satellite,...|     0.38|   e|      25421| Laurie Sanchez|8509 Todd Brooks ...|  QLD|    4345|Female| -1|-10| -9|  5

In [21]:
sdf_transformed = pipeline.fit(sdf).transform(sdf)
sdf_transformed.show(5)

+--------------------+-------+------------+------------------+--------------------+--------------------+---------+----+-----------+----------------+--------------------+-----+--------+------+---+---+---+---+---+---+---+---+---+------+---------------+--------------+-----------+---------+--------+---------------+
|            order_id|user_id|merchant_abn|      dollar_value|       merchant_name|                tags|take_rate|type|consumer_id|            name|             address|state|postcode|gender|AUS|NSW|VIC|QLD| SA| WA|TAS| NT|ACT|target|New cases / day|order_datetime|genderIndex|typeIndex|tagIndex|        tag_vec|
+--------------------+-------+------------+------------------+--------------------+--------------------+---------+----+-----------+----------------+--------------------+-----+--------+------+---+---+---+---+---+---+---+---+---+------+---------------+--------------+-----------+---------+--------+---------------+
|00085ae9-13bb-477...|   1215| 63685007785|160.82357068263116

- figure out that tag_vec

In [ ]:
SAMPLE_SIZE = 0.05
sdf_transformed = sdf_transformed.sample(SAMPLE_SIZE, seed=0)

In [8]:
df_transformed = sdf_transformed.toPandas()
df_transformed

,order_id,user_id,merchant_abn,dollar_value,merchant_name,tags,take_rate,type,consumer_id,name,...,WA,TAS,NT,ACT,target,New cases / day,order_datetime,tagIndex,typeIndex,tag_vec
0,00983e29-dbc5-4146-926c-5e533e0083bb,21133,96152467973,13.866520,Rhoncus Donec Associates,tent and awning shops,4.45,b,406997,Nicholas Miles,...,16,7,14,2,8,"1,621",2021-11-26,0.0,1.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,01fa7d0e-faf8-4447-92ef-57a975d93344,13268,41251795489,9.247176,Ultricies Sem Limited,"opticians, optical goods, and eyeglasses",2.91,c,1301967,Kristi Heath,...,16,7,14,2,-9,"1,621",2021-11-26,6.0,2.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
2,05845303-21e7-4ee4-b5ea-0e818720b2bd,8991,39112692726,55.708185,Mus Proin LLC,health and beauty spas,4.95,b,1112295,Michael Drake,...,16,7,14,2,-10,"1,621",2021-11-26,16.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0837ef49-f2f6-4d34-8e98-987d16b1d609,9502,19933438190,7.954491,Dui Nec Corporation,tent and awning shops,2.22,c,689299,Gary Davis,...,16,7,14,2,5,"1,621",2021-11-26,0.0,2.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,09964504-080e-4683-ab37-8e1b2325b78b,5588,89502033586,85.271923,Adipiscing Enim LLC,"digital goods: books, movies, music",5.98,a,763985,Scott Phelps,...,16,7,14,2,-9,"1,621",2021-11-26,2.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216953,55a8f520-2e5f-43f0-815d-94d2a5dadf00,22401,64403598239,39.465698,Lobortis Ultrices Company,"music shops - musical instruments, pianos, and...",6.31,a,1051103,Jamie Schultz,...,12,5,6,-5,12,"9,996",2021-12-25,14.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
216954,9ddf2211-389f-49f2-b925-79511b73a4c4,8432,63290521567,12.356789,Vehicula Pellentesque Corporation,artist supply and craft shops,6.48,a,283686,Brian Adams,...,12,5,6,-5,3,"9,996",2021-12-25,10.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
216955,b00192c8-4bf2-4dac-adb7-f99faec0174f,595,14492521225,27.610677,Praesent Eu Associates,tent and awning shops,2.50,c,653640,Gina Fernandez,...,12,5,6,-5,-9,"9,996",2021-12-25,0.0,2.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
216956,c0bb0b7c-a113-45bb-967c-e33db409130a,20998,15069598461,141.286621,Parturient Institute,"gift, card, novelty, and souvenir shops",4.07,b,948386,Sarah Ingram,...,12,5,6,-5,2,"9,996",2021-12-25,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
